
# TP3 – Analyse de données TLC avec Apache Spark
**Étudiante :** Melissa Moya  
**Cours :** SCI 1017  
**Date :** 10 janvier 2026



---



# 1. Configuration et initialisation

In [1]:
# Q1.1  — Configuration et initialisation (0.5pts)

# Installation de PySpark et GraphFrames
!pip install -q pyspark==3.5.0
!pip install -q graphframes


# Importation de SparkSession
from pyspark.sql import SparkSession


# Initialisation de la SparkSession avec GraphFrames
spark = (
    SparkSession.builder
    .appName("TP3_TLC")
    .config("spark.jars.packages", "graphframes:graphframes:0.8.3-spark3.5-s_2.12")
    .getOrCreate()
)


# Vérifications de l'environnement
print("Version de Spark :", spark.version)
print("GraphFrames importé avec succès")
spark

Version de Spark : 3.5.0
GraphFrames importé avec succès


# 2. Ingestion des données

In [2]:
# Q2.1 — Téléchargement des données taxis (0.5pts)
!wget -q https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-03.parquet

# Q2.2 — Télécharger les données des Zones (0.5pts)
!wget -q https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv

# Vérifier la présence des fichiers
!ls -lh yellow_tripdata_2024-03.parquet taxi_zone_lookup.csv

# Charger les données dans des DataFrames Spark
df_taxi = spark.read.parquet("yellow_tripdata_2024-03.parquet")
df_zones = spark.read.csv("taxi_zone_lookup.csv", header=True, inferSchema=True)


-rw-r--r-- 1 root root 13K Feb 22  2024 taxi_zone_lookup.csv
-rw-r--r-- 1 root root 58M May 22  2024 yellow_tripdata_2024-03.parquet


# 3. Exploration des données


In [3]:
# Importation
from IPython.display import HTML, display


# Q3.1 — Lecture et affichage des données Taxi (1pts)
print("\nQ3.1 — Lecture et affichage des données Taxi (1pts)\n")

# Lire les données dans un DataFrame : df_taxi
print("Schéma du DataFrame df_taxi :")
df_taxi.printSchema()


# Afficher les données (les 20 premières lignes)
print("\nTableau 1 — Aperçu des 20 premiers trajets :")
display(HTML(df_taxi.limit(20).toPandas().to_html(index=False)))


# Afficher le nombre total d’enregistrement
print("\nNombre total d'enregistrements dans df_taxi :", df_taxi.count())


# Q3.2 — Lecture et affichage des données Zones (1pts)
print("\nQ3.2 — Lecture et affichage des données Zones (1pts)\n")

# Lire les données dans un DataFrame : df_zones
print("Schéma du DataFrame df_zones :")
df_zones.printSchema()


# Afficher la colonne « Borough » sans doublons
print("\nTableau 2 — Borough distincts dans df_zones :")
display(HTML(df_zones.select("Borough").distinct().orderBy("Borough").toPandas().to_html(index=False)))


Q3.1 — Lecture et affichage des données Taxi (1pts)

Schéma du DataFrame df_taxi :
root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)


Tableau 1 — Aperçu des 20 premiers traje

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
1,2024-03-01 00:18:51,2024-03-01 00:23:45,0,1.30,1,N,142,239,1,8.6,3.5,0.5,2.70,0.0,1.0,16.30,2.5,0.0
1,2024-03-01 00:26:00,2024-03-01 00:29:06,0,1.10,1,N,238,24,1,7.2,3.5,0.5,3.00,0.0,1.0,15.20,2.5,0.0
2,2024-03-01 00:09:22,2024-03-01 00:15:24,1,0.86,1,N,263,75,2,7.9,1.0,0.5,0.00,0.0,1.0,10.40,0.0,0.0
2,2024-03-01 00:33:45,2024-03-01 00:39:34,1,0.82,1,N,164,162,1,7.9,1.0,0.5,1.29,0.0,1.0,14.19,2.5,0.0
1,2024-03-01 00:05:43,2024-03-01 00:26:22,0,4.90,1,N,263,7,2,25.4,3.5,0.5,0.00,0.0,1.0,30.40,2.5,0.0
2,2024-03-01 00:50:42,2024-03-01 01:10:40,1,5.04,1,N,238,159,2,25.4,1.0,0.5,0.00,0.0,1.0,27.90,0.0,0.0
2,2024-03-01 00:08:23,2024-03-01 00:17:53,1,2.15,1,N,161,141,1,12.1,1.0,0.5,5.13,0.0,1.0,22.23,2.5,0.0
2,2024-03-01 00:24:58,2024-03-01 00:30:31,1,1.10,1,N,236,237,1,8.6,1.0,0.5,2.04,0.0,1.0,15.64,2.5,0.0
2,2024-03-01 00:49:40,2024-03-01 01:01:25,1,2.78,1,N,161,114,1,14.9,1.0,0.5,2.00,0.0,1.0,21.90,2.5,0.0
1,2024-03-01 00:21:43,2024-03-01 00:24:44,1,0.30,1,N,237,141,2,5.1,3.5,0.5,0.00,0.0,1.0,10.10,2.5,0.0



Nombre total d'enregistrements dans df_taxi : 3582628

Q3.2 — Lecture et affichage des données Zones (1pts)

Schéma du DataFrame df_zones :
root
 |-- LocationID: integer (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)


Tableau 2 — Borough distincts dans df_zones :


Borough
Bronx
Brooklyn
EWR
Manhattan
N/A
Queens
Staten Island
Unknown


# 4. Transformation des données


In [4]:
# Importation
from pyspark.sql.functions import col, when, unix_timestamp
from IPython.display import HTML, display


# Q4.1 — Filtrage des données (2pts)
# Règle : supprimer les trajets où fare_amount < 0 OU trip_distance < 0

print("\nQ4.1 — Filtrage des données (2pts)\n")

df_taxi_filtered = df_taxi.filter(
    (col("fare_amount") >= 0) &
    (col("trip_distance") >= 0)
)

print(" - Enregistrements supprimés :", df_taxi.count() - df_taxi_filtered.count())
print(" - Enregistrements restants  :", df_taxi_filtered.count())


# Q4.2 — Convection de type chaine de caractère-entier (2pts)
print("\nQ4.2 — Convection de type chaine de caractère-entier (2pts)\n")

print("Tableau 3 — Valeurs distinctes AVANT transformation :")
display(HTML(
    df_taxi_filtered.select("store_and_fwd_flag")
    .distinct()
    .toPandas()
    .to_html(index=False)
))

df_taxi_encoded = df_taxi_filtered.withColumn(
    "store_and_fwd_flag",
    when(col("store_and_fwd_flag") == "Y", 1).otherwise(0)
)

print("\nTableau 4 — Valeurs distinctes APRÈS transformation :")
display(HTML(
    df_taxi_encoded.select("store_and_fwd_flag")
    .distinct()
    .toPandas()
    .to_html(index=False)
))


# Q4.3 — Calcul de la durée de trajet (2pts)
print("\nQ4.3 — Calcul de la durée de trajet (2pts)\n")

# trip_duration = epoch(dropoff) - epoch(pickup)
df_taxi_final = df_taxi_encoded.withColumn(
    "trip_duration",
    unix_timestamp(col("tpep_dropoff_datetime")) - unix_timestamp(col("tpep_pickup_datetime"))
)

print("Tableau 5 — Statistiques descriptives de trip_duration :")
display(HTML(
    df_taxi_final.select("trip_duration")
    .describe()
    .toPandas()
    .to_html(index=False)
))


# Observation
nb_neg = df_taxi_final.filter(col("trip_duration") < 0).count()
print(
    f"\nNote : {nb_neg} trajets présentent une durée négative (anomalie d'horodatage). "
    "Comme ce cas n’est pas couvert par les consignes du TP, aucune suppression supplémentaire n’a été effectuée."
)


Q4.1 — Filtrage des données (2pts)

 - Enregistrements supprimés : 58464
 - Enregistrements restants  : 3524164

Q4.2 — Convection de type chaine de caractère-entier (2pts)

Tableau 3 — Valeurs distinctes AVANT transformation :


store_and_fwd_flag
Y
N
None



Tableau 4 — Valeurs distinctes APRÈS transformation :


store_and_fwd_flag
1
0



Q4.3 — Calcul de la durée de trajet (2pts)

Tableau 5 — Statistiques descriptives de trip_duration :


summary,trip_duration
count,3524164
mean,1003.4490301813423
stddev,2045.6883232403393
min,-2214
max,545553



Note : 117 trajets présentent une durée négative (anomalie d'horodatage). Comme ce cas n’est pas couvert par les consignes du TP, aucune suppression supplémentaire n’a été effectuée.


# 5. Analyse des données

In [5]:
# Importation
from pyspark.sql.functions import col
from IPython.display import HTML, display
from graphframes import GraphFrame


# Q5.1 — Création de vues temporaires (0.5pts)
df_taxi_final.createOrReplaceTempView("taxi")
df_zones.createOrReplaceTempView("zones")


# Q5.2 — Calcul du nombre total de trajet par type de payement (1pts)
print("\nQ5.2 — Calcul du nombre total de trajet par type de payement (1pts)\n")

payment_stats = spark.sql("""
    SELECT payment_type, COUNT(*) AS total_trips
    FROM taxi
    GROUP BY payment_type
    ORDER BY total_trips DESC
""")

print("Tableau 6 — Nombre de trajets par type de paiement :")
display(HTML(payment_stats.toPandas().to_html(index=False)))



# Q5.3 — fficher pour chaque trajet les noms des zones de départ et arrivée (3pts)
print("\nQ5.3 — fficher pour chaque trajet les noms des zones de départ et arrivée (3pts)\n")

result_trajets = spark.sql("""
    SELECT
        z1.Zone AS Pickup_Zone,
        z2.Zone AS Dropoff_Zone,
        t.trip_distance,
        t.total_amount
    FROM taxi t
    LEFT JOIN zones z1 ON t.PULocationID = z1.LocationID
    LEFT JOIN zones z2 ON t.DOLocationID = z2.LocationID
""")

print("Tableau 7 — Aperçu de 20 trajets avec zones et montants :")
display(HTML(result_trajets.limit(20).toPandas().to_html(index=False)))


# Q5.4 — Créer un DataFrame réduit (1pts)
print("\nQ5.4 — Créer un DataFrame réduit (1pts)\n")

df_reduit = df_taxi_final.limit(10000)
print(f"Nombre d'enregistrements dans df_reduit : {df_reduit.count()}")


# Q5.5 — Créer un graphe (3pts)
print("\nQ5.5 — Créer un graphe (3pts)\n")

# Création des nœuds (vertices)
vertices = df_zones.select(
    col("LocationID").alias("id"),
    "Zone",
    "Borough"
).distinct()

# Création des arêtes (edges)
edges = df_reduit.select(
    col("PULocationID").alias("src"),
    col("DOLocationID").alias("dst"),
    "payment_type"
)

# Création du graphe
g = GraphFrame(vertices, edges)

print("Tableau 8 — Exemple de 5 vertices (zones) :")
display(HTML(g.vertices.limit(5).toPandas().to_html(index=False)))

print("\nTableau 9 — Exemple de 5 edges (trajets) :")
display(HTML(g.edges.limit(5).toPandas().to_html(index=False)))



# Q5.6 — Calculer le PageRank du graphe (2pts)
print("\nQ5.6 — Calculer le PageRank du graphe (2pts)\n")

pagerank_result = g.pageRank(resetProbability=0.15, maxIter=10)

top_zones = pagerank_result.vertices.select("id", "Zone", "pagerank") \
    .orderBy(col("pagerank").desc())

print("Tableau 10 — Top 20 zones selon le PageRank :")
display(HTML(top_zones.limit(20).toPandas().to_html(index=False)))

# Zone la plus desservie
zone_top = top_zones.first()
print(f"\n✓ Zone la plus desservie : ID {zone_top['id']} — {zone_top['Zone']}")


Q5.2 — Calcul du nombre total de trajet par type de payement (1pts)

Tableau 6 — Nombre de trajets par type de paiement :


payment_type,total_trips
1,2597067
2,467301
0,411888
4,31518
3,16390



Q5.3 — fficher pour chaque trajet les noms des zones de départ et arrivée (3pts)

Tableau 7 — Aperçu de 20 trajets avec zones et montants :


Pickup_Zone,Dropoff_Zone,trip_distance,total_amount
Lincoln Square East,Upper West Side South,1.30,16.30
Upper West Side North,Bloomingdale,1.10,15.20
Yorkville West,East Harlem South,0.86,10.40
Midtown South,Midtown East,0.82,14.19
Yorkville West,Astoria,4.90,30.40
Upper West Side North,Melrose South,5.04,27.90
Midtown Center,Lenox Hill West,2.15,22.23
Upper East Side North,Upper East Side South,1.10,15.64
Midtown Center,Greenwich Village South,2.78,21.90
Upper East Side South,Lenox Hill West,0.30,10.10



Q5.4 — Créer un DataFrame réduit (1pts)

Nombre d'enregistrements dans df_reduit : 10000

Q5.5 — Créer un graphe (3pts)

Tableau 8 — Exemple de 5 vertices (zones) :


...


Tableau 9 — Exemple de 5 edges (trajets) :


src,dst,payment_type
142,239,1
238,24,1
263,75,2
164,162,1
263,7,2



Q5.6 — Calculer le PageRank du graphe (2pts)

Tableau 10 — Top 20 zones selon le PageRank :


id,Zone,pagerank
265,Outside of NYC,12.849433
161,Midtown Center,5.147892
140,Lenox Hill East,4.843659
112,Greenpoint,4.820913
75,East Harlem South,4.195931
236,Upper East Side North,4.180394
230,Times Sq/Theatre District,3.966399
79,East Village,3.927741
248,West Farms/Bronx River,3.875517
237,Upper East Side South,3.774198



✓ Zone la plus desservie : ID 265 — Outside of NYC
